In [19]:
import numpy as np
import mahotas as mh
from mahotas.features import surf
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import *
from sklearn.cluster import MiniBatchKMeans
import glob
import os

In [26]:
all_instance_filenames = []
all_instance_targets = []

for f in glob.glob('cats-and-dogs-img/*.jpg'):
    target = 1 if 'cat' in os.path.split(f)[1] else 0
    all_instance_filenames.append(f)
    all_instance_targets.append(target)

surf_features = []
for f in all_instance_filenames:
    image = mh.imread(f, as_grey=True)
    surf_features.append(surf.surf(image)[:, 5:])
    
train_len = int(len(all_instance_filenames) * .60)
X_train_surf_features = np.concatenate(surf_features[:train_len])
X_test_surf_feautres = np.concatenate(surf_features[train_len:])
y_train = all_instance_targets[:train_len]
y_test = all_instance_targets[train_len:]

In [27]:
n_clusters = 300
estimator = MiniBatchKMeans(n_clusters=n_clusters)
estimator.fit_transform(X_train_surf_features)

array([[ 3.08007802,  3.51334157,  2.01943932, ...,  2.00254795,
         2.06895063,  1.78556662],
       [ 3.04997182,  3.47360874,  2.02386391, ...,  1.98351985,
         2.08163671,  1.7927809 ],
       [ 2.39582595,  2.81114883,  2.70033439, ...,  1.38084671,
         2.72528728,  2.42100352],
       ..., 
       [ 3.0304223 ,  3.44038124,  2.07183014, ...,  1.95996042,
         2.0988911 ,  1.77614317],
       [ 2.46314801,  2.90011525,  2.59277511, ...,  1.58573754,
         2.62027333,  2.26064262],
       [ 0.76145244,  1.30539461,  4.2296729 , ...,  1.05455941,
         4.18886229,  3.83995204]])

In [28]:
X_train = []
for instance in X_train_surf_features[:train_len]:
    clusters = estimator.predict(instance.reshape(1,-1))
    features = np.bincount(clusters)
    if len(features) < n_clusters:
        features = np.append(features, np.zeros((1, n_clusters-len(features))))
    X_train.append(features)

X_test = []
for instance in X_test_surf_feautres[train_len:]:
    clusters = estimator.predict(instance.reshape(1,-1))
    features = np.bincount(clusters)
    if len(features) < n_clusters:
        features = np.append(features, np.zeros((1, n_clusters-len(features))))
    X_test.append(features)

In [29]:

'''
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy
vectorizer = TfidfVectorizer()
X_train = vectorizer.fit_transform(numpy.array(X_train))
X_test = vectorizer.transform(X_test)
'''

clf = LogisticRegression(C=0.001, penalty='l2')
clf.fit_transform(X_train, y_train)
predictions = clf.predict(X_test)
predictions


E:\Anc\lib\site-packages\sklearn\utils\deprecation.py:70: DeprecationWarning: Function transform is deprecated; Support to use estimators as feature selectors will be removed in version 0.19. Use SelectFromModel instead.
  warnings.warn(msg, category=DeprecationWarning)


array([1, 1, 1, ..., 1, 1, 1])

In [30]:
print(classification_report(y_test, predictions))

ValueError: Found input variables with inconsistent numbers of samples: [40, 26576]